# Preparation

Import required dependencies to use Google's API

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

Define the accesses to be granted

In [2]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.topics', #See, create, and edit topics in Google Classroom.
          'https://www.googleapis.com/auth/classroom.coursework.students', # Manage coursework and grades for students
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets.readonly'] # See all your Google Sheets spreadsheets.

Use the provided `token.json` file to authenticate and access user data, if `token.json` is not available, it would be created from the provided `credentials.json` by logging into user's Google Account

In [3]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())



Call the Google Classroom API to access various methods with user's access from the credential that has been authenticated

In [4]:
service = build('classroom', 'v1', credentials=creds)

# Course
In this section, the user is going to choose which Google Classroom Course that is going to be accessed

Use the `courses().list()` method to show a list of the user's courses

In [5]:
results = service.courses().list(pageSize=5).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- Dummy ML
- Wizard Data Visualization
- Kementerian BUMN
- Automation
- Wizard Data Analytics


Input the course name (**case-insensitive**) that is going to be accessed (Example: `wizard data visualization`). If the course was found, the course's `id` will be retrieved.

In [6]:
course_input = input("Enter Course Name: ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

Dummy ML found with ID 529458923369


# Topic
In this section, the user is going to create new topics to group the courseworks and materials

Define the topics names in a list

In [7]:
topic_list = ['Capstone Project', 'Learning by Building', 
              'Quiz', 'Additional Assignment', 'Lecture Notes',
              'Material', 'Administration Deck']

Create the topics using `topics().create()` method, if the topic was already created, it would show `Requested entity already exists` error.

In [8]:
try:
  for name in topic_list:
    topic = {
        "name": name
    }
    response = service.courses().topics().create(
      courseId=course_id,
      body=topic).execute()

    print('Topic created: {} - {}'.format(response['name'], response['topicId']))

except HttpError as error:
  print(f"An error occurred: {error}")

Topic created: Capstone Project - 626685797091
Topic created: Learning by Building - 626684307432
Topic created: Quiz - 626684936065
Topic created: Additional Assignment - 626684670179
Topic created: Lecture Notes - 626685596678
Topic created: Material - 626684735979
Topic created: Administration Deck - 626683845119


Use the `topics().list()` method to show a list of the topics that have been created

In [9]:
topics = []

response = service.courses().topics().list(
    courseId=course_id).execute()
topics.extend(response.get('topic', []))

print('Topics : ')
for topic in topics:
    print(' - {} ({})'.format(topic['name'], topic['topicId']))

Topics : 
 - Administration Deck (626683845119)
 - Material (626684735979)
 - Lecture Notes (626685596678)
 - Additional Assignment (626684670179)
 - Quiz (626684936065)
 - Learning by Building (626684307432)
 - Capstone Project (626685797091)


# Administration Deck

In the section, the user is going to post courseworks and materials that belongs to the topic `Administration Deck`

To access the descriptions of each courseworks and materials, the user can use the `Classwork()` object from `assets.py` that store the material's descriptions as attribute.

In [10]:
from assets import Classwork

description = Classwork()

Retrieve the corresponding topic's id

In [11]:
for topic in topics:
    if topic['name'] == 'Administration Deck':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '529458923369',
 'topicId': '626683845119',
 'name': 'Administration Deck',
 'updateTime': '2023-09-29T11:54:59.190Z'}

Create the `Permohonan Sesi Mentoring` post

In [12]:
coursework = {
    'title': 'Permohonan Sesi Mentoring',
    'description': description.administration_deck['Permohonan Sesi Mentoring'],
    'materials': [
        {'link': {'url': 'http://bit.ly/mentoring-onsite?authuser=0'}},
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1SuU3e8hg9jfMO25kWang1MIaaePa2x-Y/view?usp=drive_web&authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Permohonan Sesi Mentoring created


Create the `How to Ask Questions` post

In [13]:
coursework = {
    'title': 'How to Ask Questions',
    'description': description.administration_deck['How to Ask Questions'],
    'materials': [
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material How to Ask Questions created


Create the `Administration Deck` post

In [14]:
coursework = {
    'title': 'Administration Deck',
    'description': description.administration_deck['Administration Deck'],
    'materials': [
        {'link': {'url': 'https://drive.google.com/file/d/1ivjryti41IekS55HSj43bC8TXLaZZrii/view?usp=drive_web&authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1vUG9v_Aie-aHW4U93FD7LKcjfxilHG5u/view?usp=drive_web&authuser=0'}}
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Administration Deck created


# Material
In the section, the user is going to post courseworks and materials that belongs to the topic `Material`

Retrieve the corresponding topic's id

In [15]:
for topic in topics:
    if topic['name'] == 'Material':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '529458923369',
 'topicId': '626684735979',
 'name': 'Material',
 'updateTime': '2023-09-29T11:54:57.685Z'}

Create the `6. M : Neural Network & Deep Learning (NN)` post

In [16]:
coursework = {
    'title': '6. M : Neural Network & Deep Learning (NN)',
    'materials': [
        {'link': {'url': 'https://bit.ly/nn_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 6. M : Neural Network & Deep Learning (NN) created


Create the `5. M : Time Series & Forecasting (TS)` post

In [17]:
coursework = {
    'title': '5. M : Time Series & Forecasting (TS)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ts_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 5. M : Time Series & Forecasting (TS) created


Create the `4. M : Unsupervised Learning (UL)` post

In [18]:
coursework = {
    'title': '4. M : Unsupervised Learning (UL)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ul_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 4. M : Unsupervised Learning (UL) created


Create the `3. M : Classification in Machine Learning II (C2)` post

In [19]:
coursework = {
    'title': '3. M : Classification in Machine Learning II (C2)',
    'materials': [
        {'link': {'url': 'https://bit.ly/cm2_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 3. M : Classification in Machine Learning II (C2) created


Create the `2. M : Classification in Machine Learning I (C1)` post

In [20]:
coursework = {
    'title': '2. M : Classification in Machine Learning I (C1)',
    'materials': [
        {'link': {'url': 'https://bit.ly/cm1_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 2. M : Classification in Machine Learning I (C1) created


Create the `1. M : Regression Model (RM)` post

In [21]:
coursework = {
    'title': '1. M : Regression Model (RM)',
    'materials': [
        {'link': {'url': 'https://bit.ly/rm_material'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 1. M : Regression Model (RM) created


# Quiz
In the section, the user is going to post courseworks and materials that belongs to the topic `Quiz`

Retrieve the corresponding topic's id

In [22]:
for topic in topics:
    if topic['name'] == 'Quiz':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '529458923369',
 'topicId': '626684936065',
 'name': 'Quiz',
 'updateTime': '2023-09-29T11:54:54.225Z'}

Create the `6. Q : Neural Network and Deep Learning (NN)` post

In [23]:
coursework = {
    'title': '6. Q : Neural Network and Deep Learning (NN)',
    'description': description.quiz['6. Q : Neural Network and Deep Learning (NN)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/quizzes/neural-network-quiz/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 6. Q : Neural Network and Deep Learning (NN) created


Create the `5. Q: Time Series & Forecasting (TSF)` post

In [24]:
coursework = {
    'title': '5. Q: Time Series & Forecasting (TSF)',
    'description': description.quiz['5. Q: Time Series & Forecasting (TSF)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/quizzes/time-series-quiz/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 5. Q: Time Series & Forecasting (TSF) created


Create the `4. Q: Unsupervised Learning (UL)` post

In [25]:
coursework = {
    'title': '4. Q: Unsupervised Learning (UL)',
    'description': description.quiz['4. Q: Unsupervised Learning (UL)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/unsupervised-machine-learning/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 4. Q: Unsupervised Learning (UL) created


Create the `3. Q: Classification in Machine Learning II (C2)` post

In [26]:
coursework = {
    'title': '3. Q: Classification in Machine Learning II (C2)',
    'description': description.quiz['3. Q: Classification in Machine Learning II (C2)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/machine-learning-classification-2/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 3. Q: Classification in Machine Learning II (C2) created


Create the `2. Q: Classification in Machine Learning I (C1)` post

In [27]:
coursework = {
    'title': '2. Q: Classification in Machine Learning I (C1)',
    'description': description.quiz['2. Q: Classification in Machine Learning I (C1)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/machine-learning-classification-1/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 2. Q: Classification in Machine Learning I (C1) created


Create the `0. Q: Prerequisite Check` post

In [28]:
coursework = {
    'title': '1. Q: Regression Model (RM)',
    'description': description.quiz['1. Q: Regression Model (RM)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/regression-models/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 1. Q: Regression Model (RM) created


# Learning by Building
In the section, the user is going to post courseworks and materials that belongs to the topic `Learning by Building`

Retrieve the corresponding topic's id

In [29]:
for topic in topics:
    if topic['name'] == 'Learning by Building':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '529458923369',
 'topicId': '626684307432',
 'name': 'Learning by Building',
 'updateTime': '2023-09-29T11:54:52.749Z'}

Create the `6: LBB: Neural Network & Deep Learning` post

In [30]:
coursework = {
    'title': '6: LBB: Neural Network & Deep Learning',
    'description': description.lbb['6: LBB: Neural Network & Deep Learning'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 6: LBB: Neural Network & Deep Learning created


Create the `2. LBB: Data Visualization` post

In [31]:
coursework = {
    'title': '5. LBB: Time Series & Forecasting',
    'description': description.lbb['5. LBB: Time Series & Forecasting'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 5. LBB: Time Series & Forecasting created


Create the `4. LBB: Unsupervised Learning` post

In [32]:
coursework = {
    'title': '4. LBB: Unsupervised Learning',
    'description': description.lbb['4. LBB: Unsupervised Learning'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 4. LBB: Unsupervised Learning created


Create the `3. LBB: Classification in Machine Learning II` post

In [33]:
coursework = {
    'title': '3. LBB: Classification in Machine Learning II',
    'description': description.lbb['3. LBB: Classification in Machine Learning II'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 3. LBB: Classification in Machine Learning II created


Create the `2. LBB: Classification in Machine Learning I` post

In [34]:
coursework = {
    'title': '2. LBB: Classification in Machine Learning I',
    'description': description.lbb['2. LBB: Classification in Machine Learning I'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 2. LBB: Classification in Machine Learning I created


Create the `1. LBB: Regression Model` post

In [35]:
coursework = {
    'title': '1. LBB: Regression Model',
    'description': description.lbb['1. LBB: Regression Model'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 1. LBB: Regression Model created


# Capstone Project
In the section, the user is going to post courseworks and materials that belongs to the topic `Capstone Project`

Retrieve the corresponding topic's id

In [36]:
for topic in topics:
    if topic['name'] == 'Capstone Project':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '529458923369',
 'topicId': '626685797091',
 'name': 'Capstone Project',
 'updateTime': '2023-09-29T11:54:51.177Z'}

Create the `Brief Capstone Data Visualization` post

In [37]:
coursework = {
    'title': 'Brief Capstone Machine Learning',
    'description': description.capstone['Brief Capstone Machine Learning'],
    'materials': [
        {'link': {'url': 'https://drive.google.com/file/d/1GdlVcBRIdQKEy_uMcjqNgmXTJly5vSFH/view?usp=drive_web&authuser=0'}},
        {'link': {'url': 'https://rpubs.com/Algoritma/ml-capstone?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment Brief Capstone Machine Learning created


Create the `Capstone Data Visualization` post

In [38]:
coursework = {
    'title': 'Capstone Machine Learning',
    'description': description.capstone['Capstone Machine Learning'],
    'materials': [
        {'link': {'url': 'https://rpubs.com/Algoritma/ml-capstone?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 36,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment Capstone Machine Learning created
